# Store KAKEN Master Documents to Local MariaDB

## Preparation

- KAKEN master data is maintained in a git repository and the latest data is available; pull the repository from https://bitbucket.org/niijp/grants_masterxml_kaken/ and sync it to your local . /grants_masterxml_kaken folder.

Store 5 master file from KAKEN below:

1. category_master_kakenhi.xml
2. field_master_kakenhi.xml
3. institution_master_kakenhi.xml
4. review_section_master_kakenhi.xml
5. section_master_kakenhi.xml

In [93]:
import configparser
import datetime
from dateutil.relativedelta import relativedelta
import networkx as nx
import numpy as np
import pandas as pd
import pymysql
from lxml import etree
from sqlalchemy import create_engine
from sqlalchemy.types import Date, Integer, String, Text

In [94]:
username= 'root'
password= 'Norika1124!'
database= 'Capstone_KAKEN'
appid = 'HhV6SqJ0yYlgkp0a887V'
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'
engine = create_engine(url, echo=True)

## 1. category_master_kakenhi

In [95]:
# Create list from xml file 
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/category_master_kakenhi.xml")
categorylist = []
for category_table in tree.iterfind("category_table"):
    for category in category_table.iterfind("category"):
        name = category.find("name[@lang='en']").text
        niicode = category.find("code[@type='nii']").text
        row = [niicode, name]
        categorylist.append(row)
# Convert list to dataframe
columns = ["category_niicode", "category_name"]
df = pd.DataFrame(categorylist, columns=columns)
# Check the dupuliaction, and if notm set it as an index
df = df.drop_duplicates()
assert not df["category_niicode"].duplicated().any(), "category_niicode is duplicated."
df = df.set_index("category_niicode")
df

,category_name
category_niicode,
6,Grant-in-Aid for Overseas Scientific Survey
1,Organization Research
2,Particular Research
3,Grant-in-Aid for Special Project Research
4,Grant-in-Aid for Co-operative Research
...,...
254,Fund for the Promotion of Joint International ...
253,Fund for the Promotion of Joint International ...
305,Grant-in-Aid for Transformative Research Areas...


In [96]:
# Drop Foreign Key for Database
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE grantaward DROP FOREIGN KEY category_niicode_1;")
except:
    pass
# Add Database
df.to_sql(
    "master_categories",
    engine,
    if_exists="replace",
    dtype={"category_niicode": Integer, "category_name": String(255)},
)

2022-12-08 20:13:03,505 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-08 20:13:03,506 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:03,508 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-12-08 20:13:03,509 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:03,510 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-12-08 20:13:03,511 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:03,514 INFO sqlalchemy.engine.Engine ALTER TABLE grantaward DROP FOREIGN KEY category_niicode_1;
2022-12-08 20:13:03,515 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:03,516 INFO sqlalchemy.engine.Engine ROLLBACK
2022-12-08 20:13:03,520 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:03,521 INFO sqlalchemy.engine.Engine [generated in 0.00078s] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_categories'}
2022-1

85

In [97]:
# Create list from xml files
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/category_master_kakenhi.xml")
categorylist = []
for category_table in tree.iterfind("category_table"):
    category_table_type = category_table.get("type")
    category_table_start_date = category_table.get("start_date")
    category_table_end_date = category_table.get("end_date")
    for category in category_table.iterfind("category"):
        category_path = category.get("path")
        category_name_ja = category.find("name[@lang='ja']").text
        category_name_en = category.find("name[@lang='en']").text
        category_niicode = category.find("code[@type='nii']").text
        try:
            category_name_yomi = category.find("name_yomi").text
        except AttributeError:
            category_name_yomi = None
        row = [
            category_table_type,
            category_table_start_date,
            category_table_end_date,
            category_path,
            category_name_ja,
            category_name_en,
            category_niicode,
            category_name_yomi
        ]
        categorylist.append(row)
# Convert list to dataframe
columns = [
    "category_table_type",
    "category_table_start_date",
    "category_table_end_date",
    "category_path",
    "category_name_ja",
    "category_name_en",
    "category_niicode",
    "category_name_yomi"
]
df = pd.DataFrame(categorylist, columns=columns)
df

,category_table_type,category_table_start_date,category_table_end_date,category_path,category_name_ja,category_name_en,category_niicode,category_name_yomi
0,kenkyu_syumoku,1965-04-01,1966-03-31,000006,海外学術調査,Grant-in-Aid for Overseas Scientific Survey,6,カイガイガクジュツチョウサ
1,kenkyu_syumoku,1965-04-01,1966-03-31,000001,機関研究,Organization Research,1,キカンケンキュウ
2,kenkyu_syumoku,1965-04-01,1966-03-31,000002,各個研究,Particular Research,2,カクコケンキュウ
3,kenkyu_syumoku,1965-04-01,1966-03-31,000003,特定研究,Grant-in-Aid for Special Project Research,3,トクテイケンキュウ
4,kenkyu_syumoku,1965-04-01,1966-03-31,000004,総合研究,Grant-in-Aid for Co-operative Research,4,ソウゴウケンキュウ
...,...,...,...,...,...,...,...,...
636,kenkyu_syumoku,2022-04-01,None,000042,特別研究員奨励費,Grant-in-Aid for JSPS Fellows,42,トクベツケンキュウインショウレイヒ
637,kenkyu_syumoku,2022-04-01,None,000254,国際共同研究加速基金(国際共同研究強化(A)),Fund for the Promotion of Joint International ...,254,None
638,kenkyu_syumoku,2022-04-01,None,000253,国際共同研究加速基金(国際共同研究強化(B)),Fund for the Promotion of Joint International ...,253,None
639,kenkyu_syumoku,2022-04-01,None,000079,国際共同研究加速基金(帰国発展研究),Fund for the Promotion of Joint International ...,79,コクサイキョウドウケンキュウカソクキキン(キコクハッテンケンキュウ)


In [98]:
#
# Convert from a format where start and end dates are recorded for each research line to a format where
# Convert to a format that lists research lines by fiscal year
#

# Replace NULL end date with current time
df["category_table_end_date_mod"] = df["category_table_end_date"]
df["category_table_end_date_mod"] = df["category_table_end_date_mod"].fillna(datetime.datetime.today())

# Get only the fiscal year from the date and the difference between the beginning and ending fiscal year
df["category_table_startfiscalyear"] = pd.to_datetime(df["category_table_start_date"]).map(lambda x: x.year)
df["category_table_endfiscalyear"] = pd.to_datetime(df["category_table_end_date_mod"]).map(lambda x: (x - relativedelta(months=3)).year)
df["delta"] = df["category_table_endfiscalyear"] - df["category_table_startfiscalyear"]

# Make a list of combinations of table_type and starting year
type_and_fiscalyear = df[["category_table_type", "category_table_startfiscalyear"]].drop_duplicates().values

# Duplicate rows for each start year and field table for the number of years applied
results = pd.DataFrame(columns=df.columns.values)
for l in type_and_fiscalyear:
    temp = df.query("category_table_type == @l[0] & category_table_startfiscalyear == @l[1]")
    multi = temp["delta"].unique()
    assert len(multi) == 1, "年度が集合内で単一ではありません"
    results = results.append([temp] * (multi[0] + 1), ignore_index=True)

# Add a sequential number to each line and add it to the starting year to obtain the applicable year.
results["cum"] = results.groupby(["category_table_type", "category_table_start_date", "category_path"]).cumcount()
results["applied_fiscalyear"] = results["category_table_startfiscalyear"] + results["cum"]

# Add a sequential number to each line and add it to the starting year to obtain the applicable year.
results = results[[
    "category_table_type",
    "category_table_start_date",
    "category_table_end_date",
    "applied_fiscalyear",
    "category_path",
    "category_niicode",
    "category_name_ja",
    "category_name_en",
    "category_name_yomi"
]]
results

/var/folders/d2/ygrgxxds7cn74zttdpjq_l6c0000gn/T/ipykernel_17110/610299430.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append([temp] * (multi[0] + 1), ignore_index=True)


,category_table_type,category_table_start_date,category_table_end_date,applied_fiscalyear,category_path,category_niicode,category_name_ja,category_name_en,category_name_yomi
0,kenkyu_syumoku,1965-04-01,1966-03-31,1965,000006,6,海外学術調査,Grant-in-Aid for Overseas Scientific Survey,カイガイガクジュツチョウサ
1,kenkyu_syumoku,1965-04-01,1966-03-31,1965,000001,1,機関研究,Organization Research,キカンケンキュウ
2,kenkyu_syumoku,1965-04-01,1966-03-31,1965,000002,2,各個研究,Particular Research,カクコケンキュウ
3,kenkyu_syumoku,1965-04-01,1966-03-31,1965,000003,3,特定研究,Grant-in-Aid for Special Project Research,トクテイケンキュウ
4,kenkyu_syumoku,1965-04-01,1966-03-31,1965,000004,4,総合研究,Grant-in-Aid for Co-operative Research,ソウゴウケンキュウ
...,...,...,...,...,...,...,...,...,...
870,kenkyu_syumoku,2022-04-01,None,2022,000042,42,特別研究員奨励費,Grant-in-Aid for JSPS Fellows,トクベツケンキュウインショウレイヒ
871,kenkyu_syumoku,2022-04-01,None,2022,000254,254,国際共同研究加速基金(国際共同研究強化(A)),Fund for the Promotion of Joint International ...,None
872,kenkyu_syumoku,2022-04-01,None,2022,000253,253,国際共同研究加速基金(国際共同研究強化(B)),Fund for the Promotion of Joint International ...,None
873,kenkyu_syumoku,2022-04-01,None,2022,000079,79,国際共同研究加速基金(帰国発展研究),Fund for the Promotion of Joint International ...,コクサイキョウドウケンキュウカソクキキン(キコクハッテンケンキュウ)


In [99]:
# Drop foreign key for db
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE grantaward DROP FOREIGN KEY category_niicode_1;")
except:
    pass



2022-12-08 20:13:05,064 INFO sqlalchemy.engine.Engine ALTER TABLE grantaward DROP FOREIGN KEY category_niicode_1;
2022-12-08 20:13:05,066 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:05,068 INFO sqlalchemy.engine.Engine ROLLBACK


In [100]:
# Add them to MariaDB
results.to_sql(
    "master_categories_fiscalyear",
    engine,
    if_exists="replace",
    dtype={"applied_fiscalyear": Integer},
)


2022-12-08 20:13:05,131 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:05,132 INFO sqlalchemy.engine.Engine [cached since 1.612s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_categories_fiscalyear'}
2022-12-08 20:13:05,136 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:13:05,138 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_categories_fiscalyear (
	`index` BIGINT, 
	category_table_type TEXT, 
	category_table_start_date TEXT, 
	category_table_end_date TEXT, 
	applied_fiscalyear INTEGER, 
	category_path TEXT, 
	category_niicode TEXT, 
	category_name_ja TEXT, 
	category_name_en TEXT, 
	category_name_yomi TEXT
)


2022-12-08 20:13:05,139 INFO sqlalchemy.engine.Engine [no key 0.00095s] {}
2022-12-08 20:13:05,243 INFO sqlalchemy.engine.Engine CREATE INDEX ix_master_categories_fiscalyear_index ON master_categories_fiscalyear (`index`)
2022-12-

875

In [101]:
# Set unique constraint
with engine.connect() as con:
    con.execute("ALTER TABLE `master_categories_fiscalyear` ADD UNIQUE(`category_table_type`, `applied_fiscalyear`, `category_niicode`);")

2022-12-08 20:13:05,608 INFO sqlalchemy.engine.Engine ALTER TABLE `master_categories_fiscalyear` ADD UNIQUE(`category_table_type`, `applied_fiscalyear`, `category_niicode`);
2022-12-08 20:13:05,609 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:05,863 INFO sqlalchemy.engine.Engine COMMIT


## 2. field_master_kakenhi

In [102]:
# Create list from XML file
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/field_master_kakenhi.xml")
fieldlist = []
for field_table in tree.iterfind("field_table"):
    field_table_type = field_table.get("type")
    field_table_start_date = field_table.get("start_date")
    field_table_end_date = field_table.get("end_date")
    # layer 1
    for field in field_table.iterfind("field"):
        field_path = field.get("path")
        field_niicode = field.find("code[@type='nii']").text
        field_name = field.find("name[@lang='en']").text
        row = [
            field_table_type,
            field_table_start_date,
            field_table_end_date,
            field_path,
            field_niicode,
            field_name,
            1,
        ]
        fieldlist.append(row)
        # layer 2
        for field in field.iterfind("field"):
            field_path = field.get("path")
            field_niicode = field.find("code[@type='nii']").text
            field_name = field.find("name[@lang='en']").text
            row = [
                field_table_type,
                field_table_start_date,
                field_table_end_date,
                field_path,
                field_niicode,
                field_name,
                2,
            ]
            fieldlist.append(row)
            # layer 3
            for field in field.iterfind("field"):
                field_path = field.get("path")
                field_niicode = field.find("code[@type='nii']").text
                field_name = field.find("name[@lang='en']").text
                row = [
                    field_table_type,
                    field_table_start_date,
                    field_table_end_date,
                    field_path,
                    field_niicode,
                    field_name,
                    3,
                ]
                fieldlist.append(row)
                # layer 4
                for field in field.iterfind("field"):
                    field_path = field.get("path")
                    field_niicode = field.find("code[@type='nii']").text
                    field_name = field.find("name[@lang='en']").text
                    row = [
                        field_table_type,
                        field_table_start_date,
                        field_table_end_date,
                        field_path,
                        field_niicode,
                        field_name,
                        4,
                    ]
                    fieldlist.append(row)
# Convert list to dataframe
columns = [
    "field_table_type",
    "field_table_start_date",
    "field_table_end_date",
    "field_path",
    "field_niicode",
    "field_name",
    "layer",
]
df = pd.DataFrame(fieldlist, columns=columns)
df

,field_table_type,field_table_start_date,field_table_end_date,field_path,field_niicode,field_name,layer
0,saimoku,1972-04-01,1973-03-31,000001,1,Literature,1
1,saimoku,1972-04-01,1973-03-31,000001000002,2,Philosophy,2
2,saimoku,1972-04-01,1973-03-31,000001000002000003,3,Philosophy,3
3,saimoku,1972-04-01,1973-03-31,000001000002000004,4,Chinese philosophy,3
4,saimoku,1972-04-01,1973-03-31,000001000002000005,5,印度哲学(含仏教学),3
...,...,...,...,...,...,...,...
9191,zantei_bunya,2017-04-01,2018-03-31,003105,3105,Human informatics and related fields,1
9192,zantei_bunya,2017-04-01,2018-03-31,003106,3106,Applied informatics and related fields,1
9193,zantei_bunya,2017-04-01,2018-03-31,003107,3107,Environmental analyses and evaluation and rela...,1
9194,zantei_bunya,2017-04-01,2018-03-31,003108,3108,Environmental conservation measure and related...,1


In [103]:

df["field_table_end_date_mod"] = df["field_table_end_date"]
df["field_table_end_date_mod"] = df["field_table_end_date_mod"].fillna(datetime.datetime.today())

df["field_table_startfiscalyear"] = pd.to_datetime(df["field_table_start_date"]).map(lambda x: x.year)
df["field_table_endfiscalyear"] = pd.to_datetime(df["field_table_end_date_mod"]).map(lambda x: (x - relativedelta(months=3)).year)
df["delta"] = df["field_table_endfiscalyear"] - df["field_table_startfiscalyear"]

type_and_fiscalyear = df[["field_table_type", "field_table_startfiscalyear"]].drop_duplicates().values

results = pd.DataFrame(columns=df.columns.values)
for l in type_and_fiscalyear:
    temp = df.query("field_table_type == @l[0] & field_table_startfiscalyear == @l[1]")
    multi = temp["delta"].unique()
    assert len(multi) == 1, "年度が集合内で単一ではありません"
    results = results.append([temp] * (multi[0] + 1), ignore_index=True)

results["cum"] = results.groupby(["field_table_type", "field_table_start_date", "field_path"]).cumcount()
results["applied_fiscalyear"] = results["field_table_startfiscalyear"] + results["cum"]

results = results[["field_table_type", "field_table_start_date", "field_table_end_date", "applied_fiscalyear", "field_path", "field_niicode", "field_name", "layer"]]
results

/var/folders/d2/ygrgxxds7cn74zttdpjq_l6c0000gn/T/ipykernel_17110/2642225208.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append([temp] * (multi[0] + 1), ignore_index=True)


,field_table_type,field_table_start_date,field_table_end_date,applied_fiscalyear,field_path,field_niicode,field_name,layer
0,saimoku,1972-04-01,1973-03-31,1972,000001,1,Literature,1
1,saimoku,1972-04-01,1973-03-31,1972,000001000002,2,Philosophy,2
2,saimoku,1972-04-01,1973-03-31,1972,000001000002000003,3,Philosophy,3
3,saimoku,1972-04-01,1973-03-31,1972,000001000002000004,4,Chinese philosophy,3
4,saimoku,1972-04-01,1973-03-31,1972,000001000002000005,5,印度哲学(含仏教学),3
...,...,...,...,...,...,...,...,...
26630,zantei_bunya,2017-04-01,2018-03-31,2017,003105,3105,Human informatics and related fields,1
26631,zantei_bunya,2017-04-01,2018-03-31,2017,003106,3106,Applied informatics and related fields,1
26632,zantei_bunya,2017-04-01,2018-03-31,2017,003107,3107,Environmental analyses and evaluation and rela...,1
26633,zantei_bunya,2017-04-01,2018-03-31,2017,003108,3108,Environmental conservation measure and related...,1


In [104]:
# Drop foreign key for db
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE grantaward DROP FOREIGN KEY field_niicode_1;")
        con.execute("ALTER TABLE grantaward_field DROP FOREIGN KEY fk_grantaward_field_field_niicode;")
        con.execute("ALTER TABLE grantaward_field DROP FOREIGN KEY fk_grantaward_field_field_path;")
except:
    pass


2022-12-08 20:13:08,429 INFO sqlalchemy.engine.Engine ALTER TABLE grantaward DROP FOREIGN KEY field_niicode_1;
2022-12-08 20:13:08,431 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:08,434 INFO sqlalchemy.engine.Engine ROLLBACK


In [105]:
# Add them to MariaDB
results.to_sql(
    "master_fields",
    engine,
    if_exists="replace",
    dtype={
        "field_table_type": String(255),
        "field_table_start_date": Date,
        "field_table_end_date": Date,
        "applied_fiscalyear": Integer,
        "field_path": String(255),
        "field_niicode": String(255),
        "field_name": String(255),
        "layer": Integer,
    },
)


2022-12-08 20:13:08,491 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:08,493 INFO sqlalchemy.engine.Engine [cached since 4.973s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_fields'}
2022-12-08 20:13:08,499 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:13:08,500 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_fields (
	`index` BIGINT, 
	field_table_type VARCHAR(255), 
	field_table_start_date DATE, 
	field_table_end_date DATE, 
	applied_fiscalyear INTEGER, 
	field_path VARCHAR(255), 
	field_niicode VARCHAR(255), 
	field_name VARCHAR(255), 
	layer INTEGER
)


2022-12-08 20:13:08,502 INFO sqlalchemy.engine.Engine [no key 0.00108s] {}
2022-12-08 20:13:08,606 INFO sqlalchemy.engine.Engine CREATE INDEX ix_master_fields_index ON master_fields (`index`)
2022-12-08 20:13:08,608 INFO sqlalchemy.engine.Engine [no key 0.00174s] {}
2022-12-08 20:13:

26635

In [106]:
# 主キーと外部キー制約、ユニーク制約を設定する
with engine.connect() as con:
    con.execute("ALTER TABLE master_fields ADD INDEX (field_niicode);")
    con.execute("ALTER TABLE master_fields ADD INDEX (field_path);")
    con.execute("ALTER TABLE `master_fields` ADD UNIQUE( `field_table_type`, `applied_fiscalyear`, `field_path`, `field_niicode`);")

2022-12-08 20:13:10,247 INFO sqlalchemy.engine.Engine ALTER TABLE master_fields ADD INDEX (field_niicode);
2022-12-08 20:13:10,249 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:10,518 INFO sqlalchemy.engine.Engine COMMIT
2022-12-08 20:13:10,521 INFO sqlalchemy.engine.Engine ALTER TABLE master_fields ADD INDEX (field_path);
2022-12-08 20:13:10,523 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:10,799 INFO sqlalchemy.engine.Engine COMMIT
2022-12-08 20:13:10,801 INFO sqlalchemy.engine.Engine ALTER TABLE `master_fields` ADD UNIQUE( `field_table_type`, `applied_fiscalyear`, `field_path`, `field_niicode`);
2022-12-08 20:13:10,803 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:11,094 INFO sqlalchemy.engine.Engine COMMIT


## 3. institution_master_kakenhi

institutions_table_groupのマスタ（2020年6月12日 NII成果担当から情報提供）

In [107]:
table_group_master = [
    [1, "国立大学"],
    [2, "公立大学"],
    [3, "私立大学"],
    [4, "短期大学"],
    [5, "大学共同利用機関法人"],
    [6, "文部科学省等施設等機関"],
    [7, "高等専門学校"],
    [8, "文部科学大臣が指定する機関（国の設置する研究所その他の機関）"],
    [9, "文部科学大臣が指定する機関（地方公共団体の設置する研究所その他の機関）"],
    [10, "文部科学大臣が指定する機関（法律により直接設立された法人）"],
    [11, "文部科学大臣が指定する機関（民法３４条の規定により設立された法人）"],
    [12, "文部科学大臣が指定する機関（民間等の研究機関）"],
    [13, "国際連合大学研究所等"],
    [0, "その他"],
]


In [108]:
# Drop foreign key for db
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE institutions DROP FOREIGN KEY institutions_ibfk_1;")
except:
    pass

2022-12-08 20:13:11,185 INFO sqlalchemy.engine.Engine ALTER TABLE institutions DROP FOREIGN KEY institutions_ibfk_1;
2022-12-08 20:13:11,188 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:11,273 INFO sqlalchemy.engine.Engine ROLLBACK


In [109]:
# Add them to MariaDB
table_group_master = pd.DataFrame(table_group_master, columns=["institutions_table_group", "institutions_table_group_name"])
table_group_master = table_group_master.set_index("institutions_table_group")
table_group_master.to_sql(
    "master_institutions_table_group",
    engine,
    if_exists="replace",
    dtype={
        "institutions_table_group": Integer,
        "institutions_table_group_name": String(255),
    },
)


2022-12-08 20:13:11,341 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:11,343 INFO sqlalchemy.engine.Engine [cached since 7.823s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_institutions_table_group'}
2022-12-08 20:13:11,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:13:11,349 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_institutions_table_group (
	institutions_table_group INTEGER, 
	institutions_table_group_name VARCHAR(255)
)


2022-12-08 20:13:11,351 INFO sqlalchemy.engine.Engine [no key 0.00107s] {}
2022-12-08 20:13:11,517 INFO sqlalchemy.engine.Engine CREATE INDEX ix_master_institutions_table_group_institutions_table_group ON master_institutions_table_group (institutions_table_group)
2022-12-08 20:13:11,518 INFO sqlalchemy.engine.Engine [no key 0.00126s] {}
2022-12-08 20:13:11,796 INFO sqlalchemy.engine.Engine COMMIT
2022-12-

14

In [110]:
# 外部キー制約を設定する
with engine.connect() as con:
    con.execute("ALTER TABLE `master_institutions_table_group` ADD PRIMARY KEY(`institutions_table_group`);")

2022-12-08 20:13:11,859 INFO sqlalchemy.engine.Engine ALTER TABLE `master_institutions_table_group` ADD PRIMARY KEY(`institutions_table_group`);
2022-12-08 20:13:11,861 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:12,099 INFO sqlalchemy.engine.Engine COMMIT


### Institution Table

In [111]:
# Create list from XML file
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/institution_master_kakenhi.xml")
institutionlist = []
for institution_table in tree.iterfind("institution_table"):
    institution_table_group = institution_table.get("group")
    for institution in institution_table.iterfind("institution"):
        institution_name_ja = institution.find("name[@lang='ja']").text
        institution_name_en = institution.find("name[@lang='en']").text
        try:
            institution_name_yomi = institution.find("name_yomi").text
        except AttributeError:
            institution_name_yomi = None
        institution_niicode = institution.find("code[@type='nii']").text
        institution_mextcode = institution.find("code[@type='mext']").text
        try:
            institution_jspscode = institution.find("code[@type='jsps']").text
        except AttributeError:
            institution_jspscode = None
        row = [
            institution_table_group,
            institution_niicode,
            institution_mextcode,
            institution_jspscode,
            institution_name_ja,
            institution_name_en,
            institution_name_yomi,
        ]
        institutionlist.append(row)
# Convert list to DF
columns = [
    "institution_table_group",
    "institution_niicode",
    "institution_mextcode",
    "institution_jspscode",
    "institution_name_ja",
    "institution_name_en",
    "institution_name_yomi",
]
df = pd.DataFrame(institutionlist, columns=columns)
# Check the dupuliaction, and if notm set it as an index
df = df.drop_duplicates()
assert not df["institution_niicode"].duplicated().any(), "category_niicode is duplicated."
df = df.set_index("institution_niicode")
df

,institution_table_group,institution_mextcode,institution_jspscode,institution_name_ja,institution_name_en,institution_name_yomi
institution_niicode,,,,,,
0013902,1,13902,13902,愛知教育大学,Aichi University of Education,アイチキョウイクダイガク
0011401,1,11401,11401,秋田大学,Akita University,アキタダイガク
0010107,1,10107,10107,旭川医科大学,Asahikawa Medical College,アサヒカワイカダイガク
0012101,1,12101,12101,茨城大学,Ibaraki University,イバラキダイガク
0011201,1,11201,11201,岩手大学,Iwate University,イワテダイガク
...,...,...,...,...,...,...
382783,0,87402,87402,熊本県産業技術センター(ものづくり室、材料・地域資源室、食品加工室),Kumamoto Industrial Research Institute,"クマモトケンサンギョウギジュツセンター(モノヅクリシツ,ザイリョウ･チイキシゲンシツ,ショク..."
382760,0,82610,None,独立行政法人国立国際医療研究センター,"Research Institute, International Medical Cent...",ドクリツギョウセイホウジンコクリツコクサイイリョウケンキュウセンター
382868,0,84605,None,奈良県農業研究開発センター,Nara Prefecture Agricultural Research and Deve...,ナラケンノウギョウケンキュウカイハツセンター


In [112]:
# Drop foreign key for db
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE grantaward DROP FOREIGN KEY institution_niicode_1;")
except:
    pass
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE institutions_standard DROP FOREIGN KEY institutions_standard_ibfk_1;")
        con.execute("ALTER TABLE institutions_standard DROP FOREIGN KEY institutions_standard_ibfk_2;")
except:
    pass


2022-12-08 20:13:17,284 INFO sqlalchemy.engine.Engine ALTER TABLE grantaward DROP FOREIGN KEY institution_niicode_1;
2022-12-08 20:13:17,287 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:17,293 INFO sqlalchemy.engine.Engine ROLLBACK
2022-12-08 20:13:17,296 INFO sqlalchemy.engine.Engine ALTER TABLE institutions_standard DROP FOREIGN KEY institutions_standard_ibfk_1;
2022-12-08 20:13:17,297 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:17,300 INFO sqlalchemy.engine.Engine ROLLBACK


In [113]:

# Add them to MariaDB
df.to_sql(
    "master_institutions",
    engine,
    if_exists="replace",
    dtype={
        "institution_table_group": Integer,
        "institution_niicode": String(7),
        "institution_mextcode": String(7),
        "institution_jspscode": String(7),
    },
)


2022-12-08 20:13:17,627 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:17,629 INFO sqlalchemy.engine.Engine [cached since 14.11s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_institutions'}
2022-12-08 20:13:17,633 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:13:17,635 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_institutions (
	institution_niicode VARCHAR(7), 
	institution_table_group INTEGER, 
	institution_mextcode VARCHAR(7), 
	institution_jspscode VARCHAR(7), 
	institution_name_ja TEXT, 
	institution_name_en TEXT, 
	institution_name_yomi TEXT
)


2022-12-08 20:13:17,637 INFO sqlalchemy.engine.Engine [no key 0.00226s] {}
2022-12-08 20:13:17,755 INFO sqlalchemy.engine.Engine CREATE INDEX ix_master_institutions_institution_niicode ON master_institutions (institution_niicode)
2022-12-08 20:13:17,756 INFO sqlalchemy.engine.Engine [no k

2991

In [114]:
# 外部キー制約を設定する
with engine.connect() as con:
    con.execute("ALTER TABLE `master_institutions` ADD CONSTRAINT institutions_ibfk_1 FOREIGN KEY (`institution_table_group`) REFERENCES `master_institutions_table_group`(`institutions_table_group`) ON DELETE RESTRICT ON UPDATE RESTRICT;")

2022-12-08 20:13:18,377 INFO sqlalchemy.engine.Engine ALTER TABLE `master_institutions` ADD CONSTRAINT institutions_ibfk_1 FOREIGN KEY (`institution_table_group`) REFERENCES `master_institutions_table_group`(`institutions_table_group`) ON DELETE RESTRICT ON UPDATE RESTRICT;
2022-12-08 20:13:18,378 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:18,707 INFO sqlalchemy.engine.Engine COMMIT


In [115]:
# Change of research institute codes (succession, absorption, merger) as if they were edges (edges) of a graph of starting and ending points for each research institute.
# Create a niicode list from the difference_list element in the XML file (note the order of before and after)
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/institution_master_kakenhi.xml")
edgelist = []
for institution in tree.iterfind("difference_list/institution"):
    niicode = institution.get("nii_code")
    # succeeded_from
    try:
        before = institution.find("succeeded_from/institution").get("nii_code")
        after = niicode
        edgelist.append([before, after])
    except AttributeError:
        pass
    # absorbed_into
    try:
        before = niicode
        after = institution.find("absorbed_into/institution").get("nii_code")
        edgelist.append([before, after])
    except AttributeError:
        pass
    # merged_from
    try:
        for i in institution.iterfind("merged_from/institution"):
            before = i.get("nii_code")
            after = niicode
            edgelist.append([before, after])
    except AttributeError:
        pass

# Directed graphing and outputting a list of routes from each node to all reachable nodes
g = nx.DiGraph()
g.add_edges_from(edgelist)
shortestpath = nx.shortest_path(g)

# Create conversion table
transitionlist = []
for dic in shortestpath.values():
    # Get a list of the longest paths (i.e., tracing back to the current institute code)
    l = list(dic.values())[-1]
    if len(l) == 1:
        # DO nothing
        pass
    if len(l) == 2:
        # Pass t 
        transitionlist.append(l)
    elif len(l) > 2:
        # Do not use middle node
        transitionlist.append([l[0], l[-1]])

# Add the same value to before and after for institute codes that did not change
transition = pd.DataFrame(transitionlist, columns=["before", "after"])
residue = pd.DataFrame(set(df.index) - set(transition.before))
residue = pd.concat([residue] * 2, axis=1)
residue.columns = ["before", "after"]
results = pd.concat([transition, residue]).reset_index(drop=True)
results

,before,after
0,382743,382968
1,0084601,0284601
2,0184601,0284601
3,0082650,382956
4,0182650,382956
...,...,...
2986,382899,382899
2987,0042501,0042501
2988,0032522,0032522
2989,0182728,0182728


In [116]:
# Add them to MariaDB
results.to_sql(
    "master_institutions_standard",
    engine,
    if_exists="replace",
    dtype={
        "before": String(7),
        "after": String(7),
    },
)


2022-12-08 20:13:19,889 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:19,890 INFO sqlalchemy.engine.Engine [cached since 16.37s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_institutions_standard'}
2022-12-08 20:13:19,895 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:13:19,897 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_institutions_standard (
	`index` BIGINT, 
	`before` VARCHAR(7), 
	after VARCHAR(7)
)


2022-12-08 20:13:19,898 INFO sqlalchemy.engine.Engine [no key 0.00152s] {}
2022-12-08 20:13:20,001 INFO sqlalchemy.engine.Engine CREATE INDEX ix_master_institutions_standard_index ON master_institutions_standard (`index`)
2022-12-08 20:13:20,005 INFO sqlalchemy.engine.Engine [no key 0.00416s] {}
2022-12-08 20:13:20,227 INFO sqlalchemy.engine.Engine COMMIT
2022-12-08 20:13:20,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-1

2991

In [117]:

# Set external key
with engine.connect() as con:
    con.execute("ALTER TABLE `master_institutions_standard` ADD CONSTRAINT institutions_standard_ibfk_1 FOREIGN KEY (`before`) REFERENCES `master_institutions`(`institution_niicode`) ON DELETE RESTRICT ON UPDATE RESTRICT;")
    con.execute("ALTER TABLE `master_institutions_standard` ADD CONSTRAINT institutions_standard_ibfk_2 FOREIGN KEY (`after`) REFERENCES `master_institutions`(`institution_niicode`) ON DELETE RESTRICT ON UPDATE RESTRICT;")

2022-12-08 20:13:20,411 INFO sqlalchemy.engine.Engine ALTER TABLE `master_institutions_standard` ADD CONSTRAINT institutions_standard_ibfk_1 FOREIGN KEY (`before`) REFERENCES `master_institutions`(`institution_niicode`) ON DELETE RESTRICT ON UPDATE RESTRICT;
2022-12-08 20:13:20,413 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:20,676 INFO sqlalchemy.engine.Engine COMMIT
2022-12-08 20:13:20,678 INFO sqlalchemy.engine.Engine ALTER TABLE `master_institutions_standard` ADD CONSTRAINT institutions_standard_ibfk_2 FOREIGN KEY (`after`) REFERENCES `master_institutions`(`institution_niicode`) ON DELETE RESTRICT ON UPDATE RESTRICT;
2022-12-08 20:13:20,680 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:20,954 INFO sqlalchemy.engine.Engine COMMIT


## 4. review_section_master_kakenhi

In [118]:
# Create list from XML file
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/review_section_master_kakenhi.xml")
review_sectionlist = []
for review_section_table in tree.iterfind("review_section_table"):
    review_section_table_type = review_section_table.get("type")
    review_section_table_start_date = review_section_table.get("start_date")
    review_section_table_end_date = review_section_table.get("end_date")
    # layer 1
    for review_section in review_section_table.iterfind("review_section"):
        review_section_path = review_section.get("path")
        review_section_niicode = review_section.find("code[@type='nii']").text
        review_section_name = review_section.find("name[@lang='en']").text
        row = [
            review_section_table_type,
            review_section_table_start_date,
            review_section_table_end_date,
            review_section_path,
            review_section_niicode,
            review_section_name,
            1,
        ]
        review_sectionlist.append(row)
        # layer 2
        for review_section in review_section.iterfind("review_section"):
            review_section_path = review_section.get("path")
            review_section_niicode = review_section.find("code[@type='nii']").text
            review_section_name = review_section.find("name[@lang='en']").text
            row = [
                review_section_table_type,
                review_section_table_start_date,
                review_section_table_end_date,
                review_section_path,
                review_section_niicode,
                review_section_name,
                2,
            ]
            review_sectionlist.append(row)
            # layer 3
            for review_section in review_section.iterfind("review_section"):
                review_section_path = review_section.get("path")
                review_section_niicode = review_section.find("code[@type='nii']").text
                review_section_name = review_section.find("name[@lang='en']").text
                row = [
                    review_section_table_type,
                    review_section_table_start_date,
                    review_section_table_end_date,
                    review_section_path,
                    review_section_niicode,
                    review_section_name,
                    3,
                ]
                review_sectionlist.append(row)
# Convert list to DF
columns = [
    "review_section_table_type",
    "review_section_table_start_date",
    "review_section_table_end_date",
    "review_section_path",
    "review_section_niicode",
    "review_section_name",
    "layer",
]
df = pd.DataFrame(review_sectionlist, columns=columns)
df

,review_section_table_type,review_section_table_start_date,review_section_table_end_date,review_section_path,review_section_niicode,review_section_name,layer
0,review_section,2018-04-01,None,000001,1,Broad Section A,1
1,review_section,2018-04-01,None,000001000002,2,"Medium-sized Section 1:Philosophy, art, and re...",2
2,review_section,2018-04-01,None,000001000002000003,3,Basic Section 01010:Philosophy and ethics-related,3
3,review_section,2018-04-01,None,000001000002000004,4,"Basic Section 01020:Chinese philosophy, Indian...",3
4,review_section,2018-04-01,None,000001000002000005,5,Basic Section 01030:Religious studies-related,3
...,...,...,...,...,...,...,...
821,review_section_kikoku,2020-04-01,None,000804,804,Chemistry,1
822,review_section_kikoku,2020-04-01,None,000805,805,Engineering,1
823,review_section_kikoku,2020-04-01,None,000806,806,Biological Science,1
824,review_section_kikoku,2020-04-01,None,000807,807,Agricultural Science,1


In [119]:

df["review_section_table_end_date_mod"] = df["review_section_table_end_date"]
df["review_section_table_end_date_mod"] = df["review_section_table_end_date_mod"].fillna(datetime.datetime.today())

df["review_section_table_startfiscalyear"] = pd.to_datetime(df["review_section_table_start_date"]).map(lambda x: x.year)
df["review_section_table_endfiscalyear"] = pd.to_datetime(df["review_section_table_end_date_mod"]).map(lambda x: (x - relativedelta(months=3)).year)
df["delta"] = df["review_section_table_endfiscalyear"] - df["review_section_table_startfiscalyear"]

type_and_fiscalyear = df[["review_section_table_type", "review_section_table_startfiscalyear"]].drop_duplicates().values

results = pd.DataFrame(columns=df.columns.values)
for l in type_and_fiscalyear:
    temp = df.query("review_section_table_type == @l[0] & review_section_table_startfiscalyear == @l[1]")
    multi = temp["delta"].unique()
    assert len(multi) == 1, "年度が集合内で単一ではありません"
    results = results.append([temp] * (multi[0] + 1), ignore_index=True)

# 
results["cum"] = results.groupby(["review_section_table_type", "review_section_table_start_date", "review_section_path"]).cumcount()
results["applied_fiscalyear"] = results["review_section_table_startfiscalyear"] + results["cum"]

results = results[[
    "review_section_table_type",
    "review_section_table_start_date",
    "review_section_table_end_date",
    "applied_fiscalyear",
    "review_section_path",
    "review_section_niicode",
    "review_section_name",
    "layer"
]]
results

/var/folders/d2/ygrgxxds7cn74zttdpjq_l6c0000gn/T/ipykernel_17110/4251085984.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append([temp] * (multi[0] + 1), ignore_index=True)


,review_section_table_type,review_section_table_start_date,review_section_table_end_date,applied_fiscalyear,review_section_path,review_section_niicode,review_section_name,layer
0,review_section,2018-04-01,None,2018,000001,1,Broad Section A,1
1,review_section,2018-04-01,None,2018,000001000002,2,"Medium-sized Section 1:Philosophy, art, and re...",2
2,review_section,2018-04-01,None,2018,000001000002000003,3,Basic Section 01010:Philosophy and ethics-related,3
3,review_section,2018-04-01,None,2018,000001000002000004,4,"Basic Section 01020:Chinese philosophy, Indian...",3
4,review_section,2018-04-01,None,2018,000001000002000005,5,Basic Section 01030:Religious studies-related,3
...,...,...,...,...,...,...,...,...
4240,review_section_kikoku,2020-04-01,None,2022,000804,804,Chemistry,1
4241,review_section_kikoku,2020-04-01,None,2022,000805,805,Engineering,1
4242,review_section_kikoku,2020-04-01,None,2022,000806,806,Biological Science,1
4243,review_section_kikoku,2020-04-01,None,2022,000807,807,Agricultural Science,1


In [120]:
# Drop foreign key for db
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE grantaward_review_section DROP FOREIGN KEY fk_grantaward_review_section_review_section_niicode;")
except:
    pass
# Add them to MariaDB
results.to_sql(
    "master_review_sections",
    engine,
    if_exists="replace",
    dtype={
        "review_section_table_type": String(255),
        "review_section_table_start_date": Date,
        "review_section_table_end_date": Date,
        "applied_fiscalyear": Integer,
        "review_section_path": String(255),
        "review_section_niicode": Integer,
        "layer": Integer,
    },
)


2022-12-08 20:13:21,832 INFO sqlalchemy.engine.Engine ALTER TABLE grantaward_review_section DROP FOREIGN KEY fk_grantaward_review_section_review_section_niicode;
2022-12-08 20:13:21,837 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:21,842 INFO sqlalchemy.engine.Engine ROLLBACK
2022-12-08 20:13:21,851 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:21,855 INFO sqlalchemy.engine.Engine [cached since 18.33s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_review_sections'}
2022-12-08 20:13:21,862 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:13:21,865 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_review_sections (
	`index` BIGINT, 
	review_section_table_type VARCHAR(255), 
	review_section_table_start_date DATE, 
	review_section_table_end_date DATE, 
	applied_fiscalyear INTEGER, 
	review_section_path VARCHAR(255), 
	review_sectio

4245

In [122]:

# 主キーと外部キー制約を設定する
with engine.connect() as con:
    con.execute("ALTER TABLE master_review_sections ADD INDEX (review_section_niicode);")
    con.execute("ALTER TABLE `master_review_sections` ADD UNIQUE( `review_section_table_type`, `applied_fiscalyear`, `review_section_path`, `review_section_niicode`);")

2022-12-08 20:13:42,279 INFO sqlalchemy.engine.Engine ALTER TABLE master_review_sections ADD INDEX (review_section_niicode);
2022-12-08 20:13:42,283 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:42,564 INFO sqlalchemy.engine.Engine COMMIT
2022-12-08 20:13:42,566 INFO sqlalchemy.engine.Engine ALTER TABLE `master_review_sections` ADD UNIQUE( `review_section_table_type`, `applied_fiscalyear`, `review_section_path`, `review_section_niicode`);
2022-12-08 20:13:42,567 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:42,816 INFO sqlalchemy.engine.Engine COMMIT


## 5. section_master_kakenhi

In [123]:
# Create list from XML file
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/section_master_kakenhi.xml")
sectionlist = []
for section_table in tree.iterfind("section_table"):
    for section in section_table.iterfind("section"):
        section_name = section.find("name[@lang='en']").text
        section_niicode = section.find("code[@type='nii']").text
        row = [section_niicode, section_name]
        sectionlist.append(row)
# Convert list to dataframe
columns = ["section_niicode", "section_name"]
df = pd.DataFrame(sectionlist, columns=columns)
# Check the dupuliaction, and if notm set it as an index
df = df.drop_duplicates()
assert not df["section_niicode"].duplicated().any(), "section_niicode is duplicated."
df = df.set_index('section_niicode')
df

,section_name
section_niicode,
1,本調査
2,成果とりまとめ
3,総括
4,学術定期刊行物
5,学術図書
6,二次刊行物
7,現地調査
8,調査総括
9,Special Cancer Research


In [128]:
# Drop foreign key for db
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE grantaward DROP FOREIGN KEY section_niicode_1;")
except:
    pass
# Add them to MariaDB
df.to_sql("master_sections", engine, if_exists="replace", dtype={"section_niicode": Integer})

2022-12-08 20:14:19,402 INFO sqlalchemy.engine.Engine ALTER TABLE grantaward DROP FOREIGN KEY section_niicode_1;
2022-12-08 20:14:19,403 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:14:19,406 INFO sqlalchemy.engine.Engine ROLLBACK
2022-12-08 20:14:19,411 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:14:19,412 INFO sqlalchemy.engine.Engine [cached since 75.89s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_sections'}
2022-12-08 20:14:19,416 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:14:19,419 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_sections (
	`index` BIGINT, 
	section_table_type TEXT, 
	section_table_start_date TEXT, 
	section_table_end_date TEXT, 
	section_niicode INTEGER, 
	section_mextcode TEXT, 
	section_name_ja TEXT, 
	section_name_en TEXT, 
	section_name_yomi TEXT, 
	section_table_end_date_mod TEXT, 
	section_ta

89

In [125]:
# Create list from XML file
tree = etree.parse("/Users/norika_machome/GitHub/Capstone_KAKEN/External_Sources/grants_masterxml_kaken/section_master_kakenhi.xml")
sectionlist = []
for section_table in tree.iterfind("section_table"):
    section_table_type = section_table.get("type")
    section_table_start_date = section_table.get("start_date")
    section_table_end_date = section_table.get("end_date")
    for section in section_table.iterfind("section"):
        section_niicode = section.find("code[@type='nii']").text
        try:
            section_mextcode = section.find("code[@type='mext']").text
        except AttributeError:
            section_mextcode = None
        section_name_ja = section.find("name[@lang='ja']").text
        section_name_en = section.find("name[@lang='en']").text
        section_name_yomi = section.find("name_yomi").text
        row = [
            section_table_type,
            section_table_start_date,
            section_table_end_date,
            section_niicode,
            section_mextcode,
            section_name_ja,
            section_name_en,
            section_name_yomi,
        ]
        sectionlist.append(row)
# Convert list to dataframe
columns = [
    "section_table_type",
    "section_table_start_date",
    "section_table_end_date",
    "section_niicode",
    "section_mextcode",
    "section_name_ja",
    "section_name_en",
    "section_name_yomi",
]
df = pd.DataFrame(sectionlist, columns=columns)
df

,section_table_type,section_table_start_date,section_table_end_date,section_niicode,section_mextcode,section_name_ja,section_name_en,section_name_yomi
0,kakenhi_kubun,1973-04-01,1974-03-31,1,None,本調査,本調査,ホンチョウサ
1,kakenhi_kubun,1973-04-01,1974-03-31,2,None,成果とりまとめ,成果とりまとめ,セイカトリマトメ
2,kakenhi_kubun,1973-04-01,1974-03-31,3,None,総括,総括,ソウカツ
3,kakenhi_kubun,1973-04-01,1974-03-31,4,None,学術定期刊行物,学術定期刊行物,ガクジュツテイキカンコウブツ
4,kakenhi_kubun,1973-04-01,1974-03-31,5,None,学術図書,学術図書,ガクジュツトショ
...,...,...,...,...,...,...,...,...
84,kakenhi_kubun,2014-04-01,None,17,01,一般,一般,イッパン
85,kakenhi_kubun,2014-04-01,None,23,21,国内,国内,コクナイ
86,kakenhi_kubun,2014-04-01,None,24,22,外国,外国,ガイコク
87,kakenhi_kubun,2014-04-01,None,25,04,海外学術,海外学術,カイガイガクジュツ


In [126]:
#
# Convert from a format where start and end dates are recorded for each application category to a format where
# Convert to a format where application categories are listed by fiscal year.
# # Replace NULLs in the end date with the current time.

# Replace NULLs in the end date with the current time.
df["section_table_end_date_mod"] = df["section_table_end_date"]
df["section_table_end_date_mod"] = df["section_table_end_date_mod"].fillna(datetime.datetime.today())

# Get only the fiscal year from the date and the difference between the beginning and ending fiscal year
df["section_table_startfiscalyear"] = pd.to_datetime(df["section_table_start_date"]).map(lambda x: x.year)
df["section_table_endfiscalyear"] = pd.to_datetime(df["section_table_end_date_mod"]).map(lambda x: (x - relativedelta(months=3)).year)
df["delta"] = df["section_table_endfiscalyear"] - df["section_table_startfiscalyear"]

# Make a list of combinations of table_type and starting year
type_and_fiscalyear = df[["section_table_type", "section_table_startfiscalyear"]].drop_duplicates().values

# Duplicate rows for each start year and field table for the number of years applied
results = pd.DataFrame(columns=df.columns.values)
for l in type_and_fiscalyear:
    temp = df.query("section_table_type == @l[0] & section_table_startfiscalyear == @l[1]")
    multi = temp["delta"].unique()
    assert len(multi) == 1, "年度が集合内で単一ではありません"
    results = results.append([temp] * (multi[0] + 1), ignore_index=True)

# Duplicate and add rows for the number of years for each group. Give each row a sequential number and add it to the starting year to get the applicable year.
results["cum"] = results.groupby(["section_table_type", "section_table_start_date", "section_niicode"]).cumcount()
results["applied_fiscalyear"] = results["section_table_startfiscalyear"] + results["cum"]

# Leave only the columns to be stored in the database
results = results[[
    "section_table_type",
    "section_table_start_date",
    "section_table_end_date",
    "applied_fiscalyear",
    "section_niicode",
    "section_mextcode",
    "section_name_ja",
    "section_name_en",
    "section_name_yomi",
]]
results

/var/folders/d2/ygrgxxds7cn74zttdpjq_l6c0000gn/T/ipykernel_17110/1828043279.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append([temp] * (multi[0] + 1), ignore_index=True)


,section_table_type,section_table_start_date,section_table_end_date,applied_fiscalyear,section_niicode,section_mextcode,section_name_ja,section_name_en,section_name_yomi
0,kakenhi_kubun,1973-04-01,1974-03-31,1973,1,None,本調査,本調査,ホンチョウサ
1,kakenhi_kubun,1973-04-01,1974-03-31,1973,2,None,成果とりまとめ,成果とりまとめ,セイカトリマトメ
2,kakenhi_kubun,1973-04-01,1974-03-31,1973,3,None,総括,総括,ソウカツ
3,kakenhi_kubun,1973-04-01,1974-03-31,1973,4,None,学術定期刊行物,学術定期刊行物,ガクジュツテイキカンコウブツ
4,kakenhi_kubun,1973-04-01,1974-03-31,1973,5,None,学術図書,学術図書,ガクジュツトショ
...,...,...,...,...,...,...,...,...,...
281,kakenhi_kubun,2014-04-01,None,2022,17,01,一般,一般,イッパン
282,kakenhi_kubun,2014-04-01,None,2022,23,21,国内,国内,コクナイ
283,kakenhi_kubun,2014-04-01,None,2022,24,22,外国,外国,ガイコク
284,kakenhi_kubun,2014-04-01,None,2022,25,04,海外学術,海外学術,カイガイガクジュツ


In [127]:
# Drop foreign key for db
try:
    with engine.connect() as con:
        con.execute("ALTER TABLE grantaward DROP FOREIGN KEY section_niicode_1;")
except:
    pass

# Add them to MariaDB
results.to_sql("master_sections_fiscalyear", engine, if_exists="replace", dtype={"section_niicode": Integer})

2022-12-08 20:13:55,862 INFO sqlalchemy.engine.Engine ALTER TABLE grantaward DROP FOREIGN KEY section_niicode_1;
2022-12-08 20:13:55,864 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-08 20:13:55,866 INFO sqlalchemy.engine.Engine ROLLBACK
2022-12-08 20:13:55,872 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-08 20:13:55,872 INFO sqlalchemy.engine.Engine [cached since 52.35s ago] {'table_schema': 'Capstone_KAKEN', 'table_name': 'master_sections_fiscalyear'}
2022-12-08 20:13:55,876 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-08 20:13:55,878 INFO sqlalchemy.engine.Engine 
CREATE TABLE master_sections_fiscalyear (
	`index` BIGINT, 
	section_table_type TEXT, 
	section_table_start_date TEXT, 
	section_table_end_date TEXT, 
	applied_fiscalyear BIGINT, 
	section_niicode INTEGER, 
	section_mextcode TEXT, 
	section_name_ja TEXT, 
	section_name_en TEXT, 
	section_name_yomi TE

286